# ML.Net - Fraudulent Classifier Data Exploration

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

Installed package XPlot.Plotly version 3.0.1

In [2]:
using System.IO;
using System.Net.Http;

string dataPath = "data.csv";
if (!File.Exists(dataPath))
{
    var contents = new HttpClient().GetStringAsync("http://bit.ly/2NDquHE").Result;       
    File.WriteAllText(dataPath, contents);
}

In [3]:
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

public class Transaction
{
    [ColumnName("step"), LoadColumn(0)]
    public float Step { get; set; }

    [ColumnName("type"), LoadColumn(1)]
    public string Type { get; set; }

    [ColumnName("amount"), LoadColumn(2)]
    public float Amount { get; set; }

    [ColumnName("nameOrig"), LoadColumn(3)]
    public string NameOrig { get; set; }

    [ColumnName("oldbalanceOrg"), LoadColumn(4)]
    public float OldbalanceOrg { get; set; }

    [ColumnName("newbalanceOrig"), LoadColumn(5)]
    public float NewbalanceOrig { get; set; }

    [ColumnName("nameDest"), LoadColumn(6)]
    public string NameDest { get; set; }

    [ColumnName("oldbalanceDest"), LoadColumn(7)]
    public float OldbalanceDest { get; set; }

    [ColumnName("newbalanceDest"), LoadColumn(8)]
    public float NewbalanceDest { get; set; }

    [ColumnName("isFraud"), LoadColumn(9)]
    public bool IsFraud { get; set; }

    [ColumnName("isFlaggedFraud"), LoadColumn(10)]
    public float IsFlaggedFraud { get; set; }
}

In [4]:
var mlContext = new MLContext();

var dataView = mlContext.Data.LoadFromTextFile<Transaction>(dataPath, hasHeader: true, separatorChar: ',');

In [5]:
var rows = mlContext.Data.CreateEnumerable<Transaction>(dataView, reuseRowObject: false)
                    .Take(5)
                    .ToList();

display(rows);

index,Step,Type,Amount,NameOrig,OldbalanceOrg,NewbalanceOrig,NameDest,OldbalanceDest,NewbalanceDest,IsFraud,IsFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136,160296.36,M1979787155,0,0,False,0
1,1,PAYMENT,1864.28,C1666544295,21249,19384.72,M2044282225,0,0,False,0
2,1,TRANSFER,181,C1305486145,181,0,C553264065,0,0,True,0
3,1,CASH_OUT,181,C840083671,181,0,C38997010,21182,0,True,0
4,1,PAYMENT,11668.14,C2048537720,41554,29885.86,M1230701703,0,0,False,0


In [6]:
var countFraud = dataView.GetColumn<bool>("isFraud").Count(x => x == true);
var countNotFraud = dataView.GetColumn<bool>("isFraud").Count(x => x == false);

var values = new List<int> { countFraud, countNotFraud};
var labels = new List<string> { "IsFraud", "IsNotFraud" };

var pieChart = Chart.Plot(new Graph.Pie 
{
    labels = labels,
    values = values
});

pieChart